<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7ba52ba3ded3845e07934eb2d7284d6c2536528f1392af77008563f4cf81f164
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 11:17:49
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 15.32 K (0.11%)
Current PnL: -21.02 L (-13.95%)
CY Booked + Current PnL: -9.58 L (-6.36%)
-------------------
Total profit:  1.61 L
Total loss:  -22.63 L
-------------------
Total Booked + Current PnL: 17.40 L (14.06%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.53 L (62.98%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.70,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.38,40.0,M-SC,1.00,83355.0,-17422.0,17513.0,0.16,-17.29,21.01,0.09,245.0,-0.99,0.60,9.51,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.52,45.0,H-LC,3.67,207144.0,2644.0,20652.0,1.16,1.29,9.97,11.39,10.0,0.13,1.48,10.36,X40N,ATH,PHARMA
14,BLUESTARCO,2080.00,8.25,44.0,H-MC,2.99,186920.0,22250.0,21085.0,-1.23,13.51,11.28,26.31,89.0,1.06,1.34,22.57,X40N,NTT,AC
84,VOLTAS,1530.00,-1.08,53.0,H-MC,1.18,207615.0,15873.0,21883.0,1.23,8.28,10.54,19.69,99.0,0.73,1.48,15.64,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,36.19,74.0,M-LC,7.37,221741.0,16665.0,95903.0,3.92,8.13,43.25,54.89,52.0,0.17,1.58,34.18,X5K,ATH,METALS
60,SAIL,228.00,49.16,61.0,M-MC,12.67,235087.0,10125.0,157532.0,3.64,4.50,67.01,74.53,192.0,0.06,1.68,38.21,XY24,BTT,STEEL
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.70,MH,ATH,METALS
67,SONACOMS,1006.00,-41.71,43.0,M-SC,13.81,72625.0,-28534.0,103425.0,2.41,-28.21,142.41,74.03,202.0,-0.28,0.52,2.41,AR,BTT,AUTO
55,RAJOOENG,143.10,-41.44,42.0,H-SC,16.47,88740.0,-13760.0,54362.0,2.12,-13.42,61.26,39.61,114.0,-0.25,0.63,8.38,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,24.0,M-SC,37.87,51290.0,-13716.0,164646.0,-2.29,-21.10,321.01,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-34.87,43.0,M-SC,4.40,97417.0,-31253.0,76550.0,-2.27,-24.29,78.58,35.20,219.0,-0.41,0.70,11.81,X40,NTT,FOOTWEAR
58,REPCOHOME,880.00,-59.02,56.0,H-SC,2.76,234871.0,-51634.0,308973.0,-2.14,-18.02,131.55,89.82,134.0,-0.17,1.68,24.26,XY24,NTT,FINANCE
18,CERA,9475.00,-23.63,37.0,H-SC,1.67,139414.0,-36489.0,78504.0,-1.87,-20.74,56.31,23.89,149.0,-0.46,1.00,20.40,OX40N,NTT,CERAMICS
52,PGHH,17907.65,-27.24,57.0,H-MC,2.05,210540.0,9720.0,58067.0,-1.76,4.84,27.58,33.76,80.0,0.17,1.50,10.29,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-13.34,49.0,M-LC,5.67,152086.0,1141.0,108316.0,0.42,0.76,71.22,72.51,53.0,0.01,1.09,6.82,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.54,130275.0,-12204.0,31722.0,1.15,-8.57,24.35,13.70,163.0,-0.38,0.93,53.76,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,40.0,M-SC,1.00,83355.0,-17422.0,17513.0,0.16,-17.29,21.01,0.09,245.0,-0.99,0.60,9.51,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220437.0,-49230.0,85574.0,0.13,-18.26,38.82,13.47,138.0,-0.58,1.57,12.09,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.20,53.0,H-MC,6.52,105184.0,-26651.0,68874.0,1.39,-20.22,65.48,32.03,98.0,-0.39,0.75,17.82,OX40N,NTT,IT
18,CERA,9475.0,-23.63,37.0,H-SC,1.67,139414.0,-36489.0,78504.0,-1.87,-20.74,56.31,23.89,149.0,-0.46,1.00,20.40,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.7,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,61.0,M-MC,12.67,235087.0,10125.0,157532.0,3.64,4.50,67.01,74.53,192.0,0.06,1.68,38.21,XY24,BTT,STEEL
32,HINDZINC,730.22,36.19,74.0,M-LC,7.37,221741.0,16665.0,95903.0,3.92,8.13,43.25,54.89,52.0,0.17,1.58,34.18,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.25,44.0,H-MC,2.99,186920.0,22250.0,21085.0,-1.23,13.51,11.28,26.31,89.0,1.06,1.34,22.57,X40N,NTT,AC
84,VOLTAS,1530.00,-1.08,53.0,H-MC,1.18,207615.0,15873.0,21883.0,1.23,8.28,10.54,19.69,99.0,0.73,1.48,15.64,XY25,NTT,AC
17,CAMS,5211.76,-6.49,44.0,H-SC,1.66,106263.0,4259.0,39668.0,0.40,4.18,37.33,43.06,122.0,0.11,0.76,22.01,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.70,MH,ATH,METALS
32,HINDZINC,730.22,36.19,74.0,M-LC,7.37,221741.0,16665.0,95903.0,3.92,8.13,43.25,54.89,52.0,0.17,1.58,34.18,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,11.52,193696.0,13888.0,98533.0,0.89,7.72,50.87,62.52,88.0,0.14,1.38,44.44,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.08,45.0,H-SC,8.95,128053.0,4717.0,126913.0,-0.88,3.82,99.11,106.72,119.0,0.04,0.91,28.10,AR,ATH,MISC
86,WIPRO,420.00,-13.34,49.0,M-LC,5.67,152086.0,1141.0,108316.0,0.42,0.76,71.22,72.51,53.0,0.01,1.09,6.82,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276509.0,-39333.0,151997.0,-0.33,-12.45,54.97,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205181.0,-46587.0,99800.0,0.13,-18.50,48.64,21.13,27.0,-0.47,1.47,10.37,X40,ATH,PAINTS
1,ABB,7934.00,-41.33,38.0,H-LC,7.53,245040.0,-16579.0,135801.0,-0.46,-6.34,55.42,45.57,7.0,-0.12,1.75,4.27,AR,NTT,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.50,42.0,H-LC,8.02,251480.0,-5583.0,40715.0,0.59,-2.17,16.19,13.67,24.0,-0.14,1.80,17.79,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-1.11,49.0,H-LC,1.94,158960.0,-27135.0,74616.0,-1.17,-14.58,46.94,25.51,15.0,-0.36,1.14,17.14,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
48,LTIM,7230.2,-2.89,59.0,H-LC,2.52,241132.0,-5019.0,84227.0,0.30,-2.04,34.93,32.18,16.0,-0.06,1.72,33.27,X200,ATH,IT
51,NESTLEIND,1377.0,-10.39,50.0,H-LC,2.60,275746.0,10320.0,46463.0,-0.18,3.89,16.85,21.40,11.0,0.22,1.97,10.79,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,48.0,H-LC,3.10,214952.0,4586.0,24204.0,0.77,2.18,11.26,13.68,37.0,0.19,1.54,19.06,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276509.0,-39333.0,151997.0,-0.33,-12.45,54.97,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
1,ABB,7934.00,-41.33,38.0,H-LC,7.53,245040.0,-16579.0,135801.0,-0.46,-6.34,55.42,45.57,7.0,-0.12,1.75,4.27,AR,NTT,ELECTRICAL
80,UNITDSPR,1644.00,-12.37,51.0,H-LC,6.67,229534.0,-4610.0,53229.0,0.11,-1.97,23.19,20.77,86.0,-0.09,1.64,4.81,X40N,NTT,BREWERIES
74,TCS,4406.34,-28.36,52.0,H-LC,12.18,285196.0,-60760.0,128994.0,0.22,-17.56,45.23,19.73,1.0,-0.47,2.04,5.04,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.11,49.0,H-LC,1.94,158960.0,-27135.0,74616.0,-1.17,-14.58,46.94,25.51,15.0,-0.36,1.14,17.14,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.68,31.0,H-LC,5.63,205181.0,-46587.0,99800.0,0.13,-18.50,48.64,21.13,27.0,-0.47,1.47,10.37,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.52,45.0,H-LC,3.67,207144.0,2644.0,20652.0,1.16,1.29,9.97,11.39,10.0,0.13,1.48,10.36,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.04,48.0,H-LC,3.10,214952.0,4586.0,24204.0,0.77,2.18,11.26,13.68,37.0,0.19,1.54,19.06,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,H-LC,12.18,285196.0,-60760.0,128994.0,0.22,-17.56,45.23,19.73,1.0,-0.47,2.04,5.04,X40,ATH,IT
40,INFY,2275.00,-18.16,54.0,H-LC,7.91,318861.0,5973.0,165712.0,0.13,1.91,51.97,54.87,3.0,0.04,2.28,8.69,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.27,195600.0,-4538.0,25428.0,0.06,-2.27,13.00,10.44,4.0,-0.18,1.40,3.17,X40,NTT,FMCG
83,VBL,671.64,-23.20,28.0,H-LC,10.41,276509.0,-39333.0,151997.0,-0.33,-12.45,54.97,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,38.0,H-LC,7.53,245040.0,-16579.0,135801.0,-0.46,-6.34,55.42,45.57,7.0,-0.12,1.75,4.27,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,44.0,L-SC,29.25,78893.0,-34656.0,74704.0,1.03,-30.52,94.69,35.27,268.0,-0.46,0.56,93.09,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,10.60,90255.0,-7725.0,29324.0,-1.04,-7.88,32.49,22.05,152.0,-0.26,0.64,31.19,XR,ATH,FINANCE
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.70,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.54,130275.0,-12204.0,31722.0,1.15,-8.57,24.35,13.70,163.0,-0.38,0.93,53.76,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,145053.0,-4167.0,148346.0,1.31,-2.79,102.27,96.62,208.0,-0.03,1.04,56.98,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,145053.0,-4167.0,148346.0,1.31,-2.79,102.27,96.62,208.0,-0.03,1.04,56.98,XY25,NTT,FINANCE
32,HINDZINC,730.22,36.19,74.0,M-LC,7.37,221741.0,16665.0,95903.0,3.92,8.13,43.25,54.89,52.0,0.17,1.58,34.18,X5K,ATH,METALS
69,SURYODAY,240.00,60.09,60.0,H-SC,7.97,147594.0,-31477.0,97205.0,0.07,-17.58,65.86,36.71,135.0,-0.32,1.05,46.44,XR,NTT,BANKS
50,NATIONALUM,244.55,-38.69,73.0,H-MC,14.67,122059.0,21555.0,7555.0,2.99,21.45,6.19,28.96,79.0,2.85,0.87,62.70,MH,ATH,METALS
26,GREENPANEL,537.00,232.04,58.0,M-SC,4.89,148206.0,-34872.0,113852.0,-1.16,-19.05,76.82,43.14,230.0,-0.31,1.06,39.41,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.47
2,50,75.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.96
LC    32.31
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.98
X40      14.66
X40N     11.17
XR       10.90
XY25     10.43
AR        8.14
OX40N     7.71
X200      1.72
MH        1.63
X5K       1.58
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.90
H-LC    25.76
H-MC    20.44
M-SC    13.65
M-LC     5.53
M-MC     3.02
L-SC     1.41
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.69,-7.12,42.42
IT,12.87,-16.68,79.87
FINANCE,10.49,-14.96,70.47
MISC,6.91,-18.84,84.69
BANKS,6.32,-12.57,71.79
PAINTS,5.50,-19.86,37.73
ELECTRICAL,5.33,-10.38,49.11
INSURANCE,3.83,-3.33,41.95
AC,3.51,1.47,15.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3238413.0,22
XR,1382663.0,14
AR,1189263.0,9
X40,790256.0,10
X40N,673557.0,9
OX40N,568972.0,10
XY25,479205.0,7
SR,265233.0,2
X5K,95903.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3742777.0,29
M-SC,1525105.0,17
H-MC,1291459.0,15
H-LC,1192512.0,15
M-LC,376005.0,4
M-MC,360975.0,2
L-SC,262793.0,3
L-MC,60005.0,1
L-LC,41551.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206568.0      6
M-SC       XY24       851660.0      7
H-SC       AR         809930.0      5
           XR         809024.0      7
H-MC       XY24       578870.0      4
H-LC       X40        492054.0      5
M-MC       XY24       360975.0      2
H-SC       X40N       309630.0      4
           SR         265233.0      2
           OX40N      264457.0      4
H-LC       X40N       225878.0      3
H-MC       X40        221652.0      4
H-LC       AR         210417.0      2
H-MC       XY25       177926.0      2
L-SC       XR         173579.0      2
M-LC       XY24       171786.0      2
M-SC       AR         168916.0      2
           XY25       148346.0      1
           OX40N      146427.0      4
H-MC       X40N       138049.0      2
M-SC       XR         133206.0      2
H-LC       XY25       111382.0      3
M-LC       XR         108316.0      1
H-MC       XR          98533.0      1
M-LC       X5K         95903.0      1
L-SC       OX40N       89214.0      1
H-LC       X200        84227.0      1
H-SC       MH          77935.0      1
M-SC       X40         76550.0      1
H-MC       OX40N       68874.0      1
H-LC       XY24        68554.0      1
L-MC       XR          60005.0      1
L-LC       XY25        41551.0      1
H-MC       MH           7555.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
